<a href="https://colab.research.google.com/github/llRodroll/TestGC/blob/main/Ensayo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

np.random.seed(1)              # needed to fix the simulation
T = 100                        # number of observations

b0 = 1                         # coefficients
b1 = 2
b2 = np.sqrt(b0*b1-1)
print(b2)                      # check for non-real numbers

x1 = np.random.normal(1,1.5,T) # x's
x2 = np.random.normal(1.5,2,T)
print(x1.shape)

e = np.random.normal(0,1.75,T) # error

y = b0+b1*x1+b2*x2+e           # y
print(y.shape)

1.0
(100,)
(100,)


In [ ]:
# OLS

X = np.array([np.ones(T),x1,x2]).T  # .T to transpose and X is a (T,3) matrix
print(X.shape)

A = np.linalg.inv(np.matmul(X.T,X))
B = np.matmul(X.T,y)
betas = np.matmul(A,B)
print(betas)

(100, 3)
[0.75096121 1.92778655 1.19107493]


In [ ]:
def l(par,y,x):
  # y and x will be the arguments of the function, i.e. the twist we have data
  # par column vector of coefficients with number of rows equal to k+1 where k is # of betas
  # b0 = par[0], b1 = par[1], b2 = par[2], ..., 
  # b[k+1] = s2 is the estimator for the error variance
  k = len(par)-1   # because python index starts at 0,
  betas = par[:k]  # but also, there is a difference between [:k] gives the first 3 elements but [k] gives the last
  s2 = par[k]
  
  return -(-0.5*len(y)*np.log(2*np.pi*s2) - 0.5*np.linalg.norm(y-np.dot(x,betas))/s2)

In [ ]:
from scipy.optimize import minimize

par0 = np.array([1,2,1,1.75])  # intial conditions

res = minimize(l,par0,args=(y,X),method='nelder-mead')
print(res)

 final_simplex: (array([[0.75099899, 1.92780602, 1.19105912, 0.17213367],
       [0.75092165, 1.92778394, 1.19108745, 0.17212767],
       [0.75100335, 1.92780215, 1.1910478 , 0.17213464],
       [0.75090564, 1.92783636, 1.1910679 , 0.17213253],
       [0.75095962, 1.9278213 , 1.19106084, 0.17212766]]), array([53.91928849, 53.91928849, 53.91928849, 53.9192885 , 53.91928851]))
           fun: 53.9192884864419
       message: 'Optimization terminated successfully.'
          nfev: 296
           nit: 171
        status: 0
       success: True
             x: array([0.75099899, 1.92780602, 1.19105912, 0.17213367])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in log
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from scipy.optimize import Bounds
bounds = Bounds([-np.inf,-np.inf,-np.inf,0.001], [np.inf,np.inf,np.inf,np.inf]) 

def nonl(par):
  return [par[0]*par[1]-par[2]**2]

from scipy.optimize import NonlinearConstraint


nonlinear_constraint = NonlinearConstraint(nonl, 1, 1)

res = minimize(l,par0,args=(y,X),method='trust-constr', bounds=bounds,
               constraints=nonlinear_constraint,options={'verbose': 1})

print("Value function at optimum",res.fun) 
print("Optimal control variables values",res.x)

`xtol` termination condition is satisfied.
Number of iterations: 97, function evaluations: 640, CG iterations: 210, optimality: 6.49e-07, constraint violation: 2.22e-16, execution time: 0.31 s.
Value function at optimum 54.50498761275209
Optimal control variables values [1.13114511 1.90301743 1.07358691 0.17416061]


/usr/local/lib/python3.6/dist-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)
